In [12]:
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 加载数据

In [13]:
# step1 加载文件
with open('../../data/切词.txt', 'r', encoding='utf-8') as file:
  docs = file.readlines()
print('条数: ', len(docs))
print('预览第一条: ', docs[0])

vectorizer_model = None

条数:  1000
预览第一条:  文旅文 创看 洛阳 河南省 文旅文创 发展 大会 本次 大会 安排 项目 签约 主要 方面 内容 一是 文旅 产业 项目 签约 截至 目前 梳理 重点 文旅 项目 投资总额 525.6 亿元 遴选 重大项目 进行 现场 签约 投资总额 365.8 亿元 项目 包括 文物 数字化 开发 文化 创意 园区 建设 文化 项目 涵盖 旅游 度假区 建设 旅游 酒店 民宿 打造 旅游 项目 既有 旅游 景区 开发 商旅 综合体 建设 传统 业态 项目 宇宙 基地 沉浸 演艺 业态 项目 充分体现 我省 文化 旅游 发展 特点 趋势 二是 引客 入豫 项目 签约 主要 我省 文旅 部门 文旅 企业 头部 旅行 知名 OTA 平台 重点 客源地 文旅 部门 签订 引客 入豫 协议 持续 拓展 省外 客源 市场



# 创建

In [14]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model="bert-base-chinese") # 使用bert-base-chinese
embeddings = np.load('../../data/embedding_bbc.npy') # 使用bert-base-chinese向量
print(embeddings.shape)

# 2. 创建分词模型
vectorizer_model = CountVectorizer() # 因为我们已经分好词了，所以这里不需要传入分词函数了

# 3. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=42  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 4. 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=40,
  min_samples=40,
  prediction_data=True # 一定要注意，如果后文设置了calculate_probabilities = True，一定要设置该参数 https://github.com/MaartenGr/BERTopic/issues/1103
)

# 5. 创建CountVectorizer模型
vectorizer_model = CountVectorizer(stop_words=['洛阳', '旅游', '文化'])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1000, 768)


In [15]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  calculate_probabilities = True
)

topics, probs = topic_model.fit_transform(docs, embeddings=embeddings)
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,501,-1_景区_河南_游客_城市,"[景区, 河南, 游客, 城市, 洛阳市, 中国, 历史, 活动, 河南省, 发展]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,216,0_中国_石窟_龙门石窟_世界,"[中国, 石窟, 龙门石窟, 世界, 艺术, 旅行, 景区, 位于, 河南, 历史]",[龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 单位 国家 ...
2,1,100,1_活动_景区_免费_高速,"[活动, 景区, 免费, 高速, 门票, 时间, 白云山, 地点, 期间, 体验]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
3,2,99,2_发展_城市_项目_河南,"[发展, 城市, 项目, 河南, 文旅, 活动, 中国, 国家, 游客, 建设]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
4,3,42,3_地方_很多_一天_时间,"[地方, 很多, 一天, 时间, 已经, 一点, 古城, 酒店, 西安, 感受]",[此次 四天 旅行 已经 结束 以往 不用 这次 大学 同学 一起 毕业 一年 小聚 组团 ...
5,4,42,4_旅客_郑州_高速_景区,"[旅客, 郑州, 高速, 景区, 客流, 出行, 发送, 通行, 假期, 高峰]",[国庆 长假 我市 迎来 客流 高峰 公路 铁路 发送 旅客 超过 万人次 今年 国庆 长假...


In [16]:
# UMAP可视化
reduced_embeddings = UMAP(n_neighbors=10, n_components=2).fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True, hide_annotations=True)

# 减少离群值

In [17]:
# To use this, make sure to calculate the probabilities beforehand by instantiating BERTopic with calculate_probabilities=True.
# https://maartengr.github.io/BERTopic/getting_started/outlier_reduction/outlier_reduction.html
new_topics = topic_model.reduce_outliers(docs, topics, probabilities=probs, strategy="probabilities", threshold=0.5)

In [18]:
print(new_topics.count(-1), new_topics)

395 [2, 1, -1, 0, 0, 0, 1, 3, 2, 3, -1, -1, 0, -1, -1, -1, 0, 0, 0, -1, -1, 0, -1, 0, -1, -1, 0, -1, -1, 2, 2, -1, -1, 2, -1, -1, -1, 3, -1, -1, -1, -1, 1, 3, -1, -1, -1, 2, 4, -1, 2, 0, -1, -1, 4, 0, 1, -1, -1, -1, -1, 0, 1, 0, -1, -1, 2, -1, -1, 0, 0, 3, 4, -1, 4, -1, 2, 0, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, 0, 3, -1, 1, 1, 0, 1, 0, 0, -1, -1, -1, -1, 1, -1, -1, 0, 4, -1, -1, 0, 0, -1, 0, 0, -1, -1, -1, 0, 0, 0, -1, 0, 1, -1, 0, -1, -1, 0, 0, -1, -1, -1, 0, -1, -1, 0, 0, 0, -1, -1, -1, -1, 1, 1, 0, -1, 0, -1, -1, -1, -1, -1, 1, 2, 0, 2, 1, -1, -1, 0, 0, 2, 1, 0, 0, 1, 1, -1, 2, 0, -1, -1, 2, -1, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1, 0, -1, -1, 0, -1, 2, -1, 2, 2, -1, 4, 4, 0, 2, 0, 2, 1, -1, -1, 2, -1, -1, 2, -1, 0, 2, 2, 2, -1, -1, -1, -1, 0, 2, -1, 0, -1, 4, 2, 2, 2, 0, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 0, 0, 2, 4, 2, -1, 4, 2, 0, 2, 0, 1, -1, 0, 0, 0, 1, -1, -1, -1, -1, -1, 1, 0, 4, 0, 0, 2, 1, 1, 0, 2, 2, 0, 2, 0, -1, -1, 1, -1, 2, 1, 1, 4, -1, 2, 1, 4, 2

In [19]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,501,-1_景区_河南_游客_城市,"[景区, 河南, 游客, 城市, 洛阳市, 中国, 历史, 活动, 河南省, 发展]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,216,0_中国_石窟_龙门石窟_世界,"[中国, 石窟, 龙门石窟, 世界, 艺术, 旅行, 景区, 位于, 河南, 历史]",[龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 单位 国家 ...
2,1,100,1_活动_景区_免费_高速,"[活动, 景区, 免费, 高速, 门票, 时间, 白云山, 地点, 期间, 体验]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
3,2,99,2_发展_城市_项目_河南,"[发展, 城市, 项目, 河南, 文旅, 活动, 中国, 国家, 游客, 建设]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
4,3,42,3_地方_很多_一天_时间,"[地方, 很多, 一天, 时间, 已经, 一点, 古城, 酒店, 西安, 感受]",[此次 四天 旅行 已经 结束 以往 不用 这次 大学 同学 一起 毕业 一年 小聚 组团 ...
5,4,42,4_旅客_郑州_高速_景区,"[旅客, 郑州, 高速, 景区, 客流, 出行, 发送, 通行, 假期, 高峰]",[国庆 长假 我市 迎来 客流 高峰 公路 铁路 发送 旅客 超过 万人次 今年 国庆 长假...


In [20]:
# 应用更新
topic_model.update_topics(docs, topics=new_topics, vectorizer_model=vectorizer_model)

In [21]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,395,-1_景区_河南_游客_城市,"[景区, 河南, 游客, 城市, 中国, 洛阳市, 历史, 博物馆, 河南省, 发展]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,225,0_中国_石窟_龙门石窟_世界,"[中国, 石窟, 龙门石窟, 世界, 艺术, 旅行, 位于, 景区, 河南, 历史]",[龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 单位 国家 ...
2,1,116,1_活动_景区_免费_时间,"[活动, 景区, 免费, 时间, 高速, 门票, 白云山, 地点, 广场, 体验]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
3,2,143,2_发展_河南_城市_活动,"[发展, 河南, 城市, 活动, 项目, 景区, 中国, 文旅, 建设, 游客]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
4,3,42,3_地方_很多_一天_时间,"[地方, 很多, 一天, 时间, 已经, 一点, 古城, 酒店, 西安, 感受]",[此次 四天 旅行 已经 结束 以往 不用 这次 大学 同学 一起 毕业 一年 小聚 组团 ...
5,4,79,4_景区_旅客_郑州_年票,"[景区, 旅客, 郑州, 年票, 防控, 疫情, 客流, 高速, 开放, 客运]",[国庆 长假 我市 迎来 客流 高峰 公路 铁路 发送 旅客 超过 万人次 今年 国庆 长假...


In [22]:
# UMAP可视化
reduced_embeddings = UMAP(n_neighbors=10, n_components=2).fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True, hide_annotations=True)